In [1]:
from binary_param import linear, constant
import pandas as pd
import numpy as np
from R32_bmimBF4_PR import configuration
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import Flash
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
def PR_model(data):
    """
    Function which returns initilized model.

    Args:
        data: pandas DataFrame with data

    Returns:
        initialized model
    """
    eps = 0.1
    init_kappa_2_1A = 0.07
    init_kappa_1_2A = 0.02
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})
    x = float(data['x_R32'])+eps
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp['bmimBF4'].fix(1-x)
    m.fs.state_block.mole_frac_comp['R32'].fix(x)

    # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa_A['bmimBF4','bmimBF4'].fix(0)
    m.fs.properties.PR_kappa_A['bmimBF4', 'R32'].fix(init_kappa_2_1A)
    m.fs.properties.PR_kappa_A['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_A['R32', 'bmimBF4'].fix(init_kappa_1_2A)
    m.fs.properties.PR_kappa_B['bmimBF4', 'bmimBF4'].fix(0)
    m.fs.properties.PR_kappa_B['bmimBF4', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'bmimBF4'].fix(0)
    m.fs.properties.PR_kappa_C['bmimBF4', 'bmimBF4'].fix(0)
    m.fs.properties.PR_kappa_C['bmimBF4', 'R32'].fix(0)
    m.fs.properties.PR_kappa_C['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_C['R32', 'bmimBF4'].fix(0)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp['bmimBF4'].unfix()
    m.fs.state_block.mole_frac_comp['R32'].unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data['x_R32']))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimBF4'].fix(float(data['x_bmimBF4']))
    m.fs.state_block.mole_frac_comp['R32'].fix(float(data['x_R32'])+eps)
    m.fs.state_block.mole_frac_comp['bmimBF4'].unfix()

    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['bmimBF4', 'R32'].setlb(-5)
    m.fs.properties.PR_kappa_A['bmimBF4', 'R32'].setub(5)

    m.fs.properties.PR_kappa_A['R32', 'bmimBF4'].setlb(-5)
    m.fs.properties.PR_kappa_A['R32', 'bmimBF4'].setub(5)

    # Return initialized flash model
    return m

def SSE(m, data):
    """
    returns objective function expresion.

    Args:
        m: model
        data: pandas DataFrame with data

    Returns:
        objectuve function scaled expresion
    """
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)

    return expr * 1e-6

variable_name = ["fs.properties.PR_kappa_A['bmimBF4', 'R32']",
                 "fs.properties.PR_kappa_A['R32', 'bmimBF4']"]

#     pest = parmest.Estimator(PR_model, data, variable_name, SSE, tee=True)

#     obj_value, parameters,a= pest.theta_est(calc_cov=True)

In [3]:
data = pd.read_csv('R32_bmimBF4.csv')

# print(data)


In [4]:
N = 10 #number of samples
results_params = np.zeros((N,2))
results_obj = np.zeros((N,1))

st_dev_T = 0.01 #C
st_dev_P = 9771 #MPa
st_dev_x = 0.07 #mole fraction

for i in range(N):
    print(i)
#     noise_T = np.random.normal(loc=0,scale=st_dev_T)
#     print(noise_T)
#     noise_P = np.random.normal(loc=0,scale=st_dev_P)
#     print(noise_P)
    noise_x = np.random.normal(loc=0,scale=st_dev_x)
    print(noise_x)
    data_ = data.copy()
#     data_["temperature"] += noise_T
    data_["x_bmimBF4"] += data_["x_bmimBF4"] * noise_x
#     data_["pressure"] += noise_P
    
#     data_.loc[data_['pressure'] <= 0, 'pressure'] = 1000
    data_.loc[data_['x_bmimBF4'] <= 0.0001, 'x_bmimBF4'] = 0.0001
    data_.loc[data_['x_bmimBF4'] >= 0.9999 , 'x_bmimBF4'] = 0.9999      

    try:
        pest = parmest.Estimator(PR_model, data_, variable_name, SSE, tee=False)

        obj_value, parameters = pest.theta_est()

        for k,v in parameters.items():
            print(k, "=", v)

        line = 0

        for k,v in parameters.items():
            results_params[i,line] = v
            line += 1
        results_obj[i] = obj_value

    except ValueError:
        results_params[i,0] = 'NaN'
        results_params[i,1] = 'NaN'


0
-0.012212743299022536
2021-10-28 13:12:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:12:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:13:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:13:01 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:13:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:13:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:13:05 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-10-28 13:13:06 [INFO] idaes.init.fs.state_block: Property package initializa

2021-10-28 13:14:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:14:43 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:14:45 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:14:46 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-10-28 13:14:47 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:14:49 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:14:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:14:51 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal 

2021-10-28 13:16:33 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-10-28 13:16:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:16:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:16:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:16:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:16:39 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:16:40 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:16:41 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal 

2021-10-28 13:18:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:18:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:18:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:18:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:18:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:18:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:18:25 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:18:26 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:18:27 [INFO] idaes

2021-10-28 13:19:48 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:19:49 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:19:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:19:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:19:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:19:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:19:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:19:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-10-28 13:19:58 [INFO] idaes

In [5]:
print(results_params)

[[ 4.32922561e-03 -1.06015582e-04]
 [ 3.97066849e-03 -1.03806549e-04]
 [ 4.02118011e-03 -1.04123552e-04]
 [ 5.02123638e-03 -1.07828188e-04]
 [ 3.67911041e-03 -1.01940449e-04]
 [ 4.20458364e-03 -1.05258605e-04]
 [ 5.78415251e-03 -9.79766900e-05]
 [ 3.59350992e-03 -1.01381093e-04]
 [ 4.47866032e-03 -1.06907656e-04]
 [ 5.41860563e-03 -1.02696979e-04]]


In [6]:
results = pd.DataFrame(results_params)
results.to_csv('R32_bmim_bf4_07.csv')